# **1. Library**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# **2. Utils**

## **2.1. reduce_mem_usage**

In [ ]:
%%time
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 9.78 µs


## **2.2. longest_streak**

In [ ]:
def longest_streak(arr):
    """
    https://codereview.stackexchange.com/questions/138550/count-consecutive-ones-in-a-binary-list
    """
    one_list = []
    size = 0
    for num in arr:
        if num == 1:
            one_list.append(num)
        elif num == 0 and size < len(one_list):
            size = len(one_list)
            one_list = []
    return max(size, len(one_list))

## **2.3. group_entropy**

In [ ]:
def group_entropy(df, group, subgroup, cname, value, df_feats):
  if isinstance(subgroup, list):
      full_group = [group]
      full_group.extend(subgroup)
  else:
      full_group = [group, subgroup]

  gp_1 = df.groupby(full_group)[value].count().reset_index()
  gp_1.columns = full_group + ['subgroup_cnt']

  gp_2 = df.groupby(group)[value].count().reset_index()
  gp_2.columns = [group, 'cnt']

  gp_3 = gp_2.merge(gp_1, on=group, how='left')

  gp_3['entropy'] = -np.log(gp_3['subgroup_cnt'] / gp_3['cnt']) * gp_3['subgroup_cnt'] / gp_3['cnt']
  gp_3['entropy'].fillna(0, inplace=True)

  gp_4 = gp_3.groupby(group)['entropy'].sum().reset_index()
  gp_4.columns = [group, cname]

  df_feats = df_feats.merge(gp_4, on=group, how='left')
  
  return df_feats

# **3. Analyst**

## **3.1. Pre-process**

In [ ]:
new_dir = '/content/drive/MyDrive/Colab Notebooks/viettel/data/new_merchant_transactions.csv' # change here

In [ ]:
# Read data
new = pd.read_csv()

# Reduce memory usage
new = reduce_mem_usage(new)

# Replace inf, illegal values by nan values
new.replace([-np.inf, np.inf], np.nan, inplace=True) 
new['installments'].replace(-1, np.nan, inplace=True)
new['installments'].replace(999, np.nan, inplace=True)

# Fill nan values of categorical features by using mode
new['category_2'].fillna(1, inplace=True)
new['category_3'].fillna('A', inplace=True)
new['merchant_id'].fillna('M_ID_00a6ca8a8a', inplace=True)
new['installments'].fillna(0, inplace=True)

# Encode categorical feature
cols = ['authorized_flag', 'category_1', 'category_3']
lbl_encoder = LabelEncoder()
for c in cols:
    new[c] = lbl_encoder.fit_transform(new[c].astype(str))

# Transfer data to datetime type
new['purchase_date'] = pd.to_datetime(new['purchase_date'])
new = new.sort_values('purchase_date')

# Get reference month
new['purchase_month'] = new['purchase_date'].astype(str).apply(lambda x: x[:7] + '-28')
new['reference_month'] = pd.to_datetime(new['purchase_month']) - new['month_lag'].apply(lambda x: np.timedelta64(x, 'M'))
new['reference_month'] = new['reference_month'].astype(str).apply(lambda x: x[:7])
new.drop(['purchase_month'], axis=1, inplace=True)

# One-hot categorical features: month_lag, category_2, category_3
new['month_lag=1']  = (new['month_lag'] == 1).astype(int)
new['month_lag=2'] = (new['month_lag'] == 2).astype(int)

new['category_2=1'] = (new['category_2'] == 1.).astype(int)
new['category_2=2'] = (new['category_2'] == 2.).astype(int)
new['category_2=3'] = (new['category_2'] == 3.).astype(int)
new['category_2=4'] = (new['category_2'] == 4.).astype(int)
new['category_2=5'] = (new['category_2'] == 5.).astype(int)

new['category_3=0'] = (new['category_3'] == 0).astype(int)
new['category_3=1'] = (new['category_3'] == 1).astype(int)
new['category_3=2'] = (new['category_3'] == 2).astype(int)
new['category_3=3'] = (new['category_3'] == 3).astype(int)

Mem. usage decreased to 114.20 Mb (45.5% reduction)


In [ ]:
# Normalize 'purchase_amount'
new['purchase_amount'] = new['purchase_amount'].astype(np.float64) 
new['purchase_amount'] = np.round(new['purchase_amount'] / 0.00150265118 + 497.06, 2)

In [ ]:
# Create new_feats table
new_feats = pd.DataFrame(new.groupby(['card_id']).size()).reset_index()
new_feats.columns = ['card_id', 'new_transac_count']

In [ ]:
new.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,reference_month,month_lag=1,month_lag=2,category_2=1,category_2=2,category_2=3,category_2=4,category_2=5,category_3=0,category_3=1,category_3=2,category_3=3
355331,0,C_ID_7c628841cb,69,0,1.0,1,80,M_ID_c03b62d83d,1,46.36,2017-03-01 03:24:51,1.0,9,37,2017-02,1,0,1,0,0,0,0,0,1,0,0
1371578,0,C_ID_25d399500c,19,0,1.0,1,307,M_ID_2445d76702,1,174.06,2017-03-01 11:01:06,1.0,9,19,2017-02,1,0,1,0,0,0,0,0,1,0,0
1578963,0,C_ID_e3542c52f1,199,0,0.0,0,307,M_ID_b16ae63c45,1,49.93,2017-03-01 11:27:39,4.0,14,19,2017-02,1,0,0,0,0,1,0,1,0,0,0
1073844,0,C_ID_fabd47ca44,69,0,0.0,0,80,M_ID_b35d0757d1,1,5.42,2017-03-01 11:55:11,1.0,9,37,2017-02,1,0,1,0,0,0,0,1,0,0,0
530878,0,C_ID_6f9a771d17,96,0,1.0,1,178,M_ID_b6b9b8ed67,1,549.99,2017-03-01 12:37:26,2.0,24,29,2017-02,1,0,0,1,0,0,0,0,1,0,0


In [ ]:
new_feats.head()

,card_id,new_transac_count
0,C_ID_00007093c1,2
1,C_ID_0001238066,26
2,C_ID_0001506ef0,2
3,C_ID_0001793786,31
4,C_ID_000183fdda,11


## **3.2. Feature Engineering**

### **3.2.1. info**

In [ ]:
# Count unique values for city_id, state_id, merchant_category_id, subsector_id, merchant_id 
for c in ['city', 'state', 'merchant_category', 'subsector', 'merchant']:
    new_feats['new_transac_{}_nunique'.format(c)] = new.groupby(['card_id'])['{}_id'.format(c)].nunique().values

# One-hot category_1: sum(), mean(), std()
new_feats['new_transac_category_1_1_count'] = new.groupby(['card_id'])['category_1'].sum().values
new_feats['new_transac_category_1_0_count'] = new_feats['new_transac_count'].values - new_feats['new_transac_category_1_1_count'].values
new_feats['new_transac_category_1_1_mean'] = new.groupby(['card_id'])['category_1'].mean().values
new_feats['new_transac_category_1_1_std'] = new.groupby(['card_id'])['category_1'].std().values

# One-hot category_2, category_3: sum(), mean()
for c in ['category_2=1', 'category_2=2', 'category_2=3', 'category_2=4', 'category_2=5', 
          'category_3=0', 'category_3=1', 'category_3=2', 'category_3=3']:
    new_feats['new_transac_{}_count'.format(c)] = new.groupby(['card_id'])[c].sum().values
    new_feats['new_transac_{}_mean'.format(c)]  = new.groupby(['card_id'])[c].mean().values

# Installments count
for m in ['mean', 'sum', 'max', 'min', 'std', 'skew']:
    new_feats['new_transac_installments_{}'.format(m)] = new.groupby(['card_id'])['installments'].agg([m]).values

# Monthlag count purchase_amount
new_monthsum_count = new.groupby(['card_id', 'month_lag'])['purchase_amount'].count().unstack().fillna(0.0).reset_index()
new_feats['new_transac_monthlag_count_std'] = new_monthsum_count.iloc[:, 1:].std(axis=1).values
new_feats['new_transac_monthlag_count_max'] = new_monthsum_count.iloc[:, 1:].max(axis=1).values

# Group entropy
for c in ['merchant_category_id', 'subsector_id', 'merchant_id', 'city_id', 'state_id', 'category_1', 'category_2', 'category_3', 'month_lag']:
    new_feats = group_entropy(new, 'card_id', c, 'new_transac_{}_entropy'.format(c), 'purchase_amount', new_feats)

In [ ]:
for c in new_feats.columns[new_feats.isnull().sum() != 0].values:
    new_feats[c].fillna(new_feats[c].median(), inplace=True)

In [ ]:
new.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,reference_month,month_lag=1,month_lag=2,category_2=1,category_2=2,category_2=3,category_2=4,category_2=5,category_3=0,category_3=1,category_3=2,category_3=3
355331,0,C_ID_7c628841cb,69,0,1.0,1,80,M_ID_c03b62d83d,1,46.36,2017-03-01 03:24:51,1.0,9,37,2017-02,1,0,1,0,0,0,0,0,1,0,0
1371578,0,C_ID_25d399500c,19,0,1.0,1,307,M_ID_2445d76702,1,174.06,2017-03-01 11:01:06,1.0,9,19,2017-02,1,0,1,0,0,0,0,0,1,0,0
1578963,0,C_ID_e3542c52f1,199,0,0.0,0,307,M_ID_b16ae63c45,1,49.93,2017-03-01 11:27:39,4.0,14,19,2017-02,1,0,0,0,0,1,0,1,0,0,0
1073844,0,C_ID_fabd47ca44,69,0,0.0,0,80,M_ID_b35d0757d1,1,5.42,2017-03-01 11:55:11,1.0,9,37,2017-02,1,0,1,0,0,0,0,1,0,0,0
530878,0,C_ID_6f9a771d17,96,0,1.0,1,178,M_ID_b6b9b8ed67,1,549.99,2017-03-01 12:37:26,2.0,24,29,2017-02,1,0,0,1,0,0,0,0,1,0,0


In [ ]:
new_feats.head()

,card_id,new_transac_count,new_transac_city_nunique,new_transac_state_nunique,new_transac_merchant_category_nunique,new_transac_subsector_nunique,new_transac_merchant_nunique,new_transac_category_1_1_count,new_transac_category_1_0_count,new_transac_category_1_1_mean,new_transac_category_1_1_std,new_transac_category_2=1_count,new_transac_category_2=1_mean,new_transac_category_2=2_count,new_transac_category_2=2_mean,new_transac_category_2=3_count,new_transac_category_2=3_mean,new_transac_category_2=4_count,new_transac_category_2=4_mean,new_transac_category_2=5_count,new_transac_category_2=5_mean,new_transac_category_3=0_count,new_transac_category_3=0_mean,new_transac_category_3=1_count,new_transac_category_3=1_mean,new_transac_category_3=2_count,new_transac_category_3=2_mean,new_transac_category_3=3_count,new_transac_category_3=3_mean,new_transac_installments_mean,new_transac_installments_sum,new_transac_installments_max,new_transac_installments_min,new_transac_installments_std,new_transac_installments_skew,new_transac_monthlag_count_std,new_transac_monthlag_count_max,new_transac_merchant_category_id_entropy,new_transac_subsector_id_entropy,new_transac_merchant_id_entropy,new_transac_city_id_entropy,new_transac_state_id_entropy,new_transac_category_1_entropy,new_transac_category_2_entropy,new_transac_category_3_entropy,new_transac_month_lag_entropy
0,C_ID_00007093c1,2,2,2,2,2,2,0,2,0.000000,0.000000,1,0.500000,0,0.000000,1,0.50000,0,0.0,0,0.000000,0,0.000000,2,1.000000,0,0.000000,0,0,1.000000,2.0,1.0,1.0,0.000000,0.000000,1.414214,2.0,0.693147,0.693147,0.693147,0.693147,0.693147,0.000000,0.693147,0.000000,0.000000
1,C_ID_0001238066,26,8,4,15,9,25,2,24,0.076923,0.271746,23,0.884615,0,0.000000,0,0.00000,0,0.0,3,0.115385,1,0.038462,21,0.807692,4,0.153846,0,0,1.653846,43.0,10.0,0.0,2.038476,3.365920,5.656854,17.0,2.414284,1.892616,3.204778,1.904569,0.852865,0.271189,0.357627,0.585783,0.645033
2,C_ID_0001506ef0,2,1,1,2,2,2,0,2,0.000000,0.000000,0,0.000000,0,0.000000,2,1.00000,0,0.0,0,0.000000,2,1.000000,0,0.000000,0,0.000000,0,0,0.000000,0.0,0.0,0.0,0.000000,0.000000,1.414214,2.0,0.693147,0.693147,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,C_ID_0001793786,31,7,5,21,14,31,0,31,0.000000,0.000000,17,0.548387,8,0.258065,5,0.16129,0,0.0,1,0.032258,31,1.000000,0,0.000000,0,0.000000,0,0,0.000000,0.0,0.0,0.0,0.000000,0.000000,7.778175,21.0,2.798832,2.254987,3.433987,1.508533,1.282704,0.000000,1.084073,0.000000,0.628799
4,C_ID_000183fdda,11,2,2,9,6,11,0,11,0.000000,0.000000,0,0.000000,0,0.000000,11,1.00000,0,0.0,0,0.000000,1,0.090909,6,0.545455,4,0.363636,0,0,1.545455,17.0,4.0,0.0,1.128152,1.139664,3.535534,8.0,2.098274,1.641735,2.397895,0.304636,0.304636,0.000000,0.000000,0.916465,0.585953


### **3.2.2. amount**

In [ ]:
# Purchase_amount
for m in ['sum', 'mean', 'max', 'min', 'median', 'std', 'skew']:
    new_feats['new_transac_amount_{}'.format(m)] = new.groupby(['card_id'])['purchase_amount'].agg([m]).values
new_feats['new_transac_amount_diff'] = new_feats['new_transac_amount_max'].values - new_feats['new_transac_amount_min'].values

# Total purchase amount in last 2 months
new_monthsum_amount = new.groupby(['card_id', 'month_lag'])['purchase_amount'].sum().unstack(fill_value=0.0).reset_index()
new_feats['new_transac_monthlag_last_1_amount'] = new_monthsum_amount.iloc[:, -1].values
new_feats['new_transac_monthlag_last_2_amount'] = new_monthsum_amount.iloc[:, -2].values
new_feats['new_transac_monthlag_last_2_1_amount_ratio'] = new_monthsum_amount.iloc[:, -2].values / new_monthsum_amount.iloc[:, -1].values
new_feats['new_transac_monthlag_last_2_1_amount_ratio'] = new_feats['new_transac_monthlag_last_2_1_amount_ratio'].replace([np.inf, -np.inf], np.nan)
new_feats['new_transac_monthlag_last_2_1_amount_log_ratio'] = np.log2(new_feats['new_transac_monthlag_last_2_1_amount_ratio'])

# Purchase_amount decay
tmp_df = new.groupby(['card_id']).size().reset_index()
tmp_df.columns = ['card_id', 'new_transac_count']
new = new.merge(tmp_df, on=['card_id'], how='left')

new['transac_seq_num'] = new.groupby(['card_id']).cumcount() + 1
new['transac_seq_num_desc'] = new['new_transac_count'] - new['transac_seq_num'] - 1
new['transac_decay'] = 0.8 ** new['transac_seq_num_desc'].values
new['transac_amount_decay'] = new['purchase_amount'] * new['transac_decay']
new['transac_month_decay'] = 1.2 ** new['month_lag'] + 1.
new['transac_amount_month_decay'] = new['purchase_amount'] * new['transac_month_decay']

for m in ['sum', 'mean', 'max', 'min', 'median', 'std', 'skew']:
    new_feats['new_transac_amount_decay_{}'.format(m)] = new.groupby(['card_id'])['transac_amount_decay'].agg([m]).values
    new_feats['new_transac_amount_month_decay_{}'.format(m)] = new.groupby(['card_id'])['transac_amount_month_decay'].agg([m]).values
for m in ['mean', 'sum']:
    new_feats['newtransac_decay_{}'.format(m)] = new.groupby(['card_id'])['transac_decay'].agg([m]).values

# Aggregation function on purchase_amount by using 'category_1', 'category_2', 'category_3', 'installments', 'city_id', 'merchant_category_id', 'merchant_id', 'subsector_id'
def successive_aggregates(df, field1, field2):
    t = df.groupby(['card_id', field1])[field2].mean()
    u = pd.DataFrame(t).reset_index().groupby('card_id')[field2].agg(['mean', 'min', 'max', 'std'])
    u.columns = ['new_transac_' + field1 + '_' + field2 + '_' + c for c in u.columns.values]
    u.reset_index(inplace=True)
    return u

tmp_df_1 = successive_aggregates(new, 'category_1', 'purchase_amount')
tmp_df_2 = successive_aggregates(new, 'installments', 'purchase_amount')
tmp_df_3 = successive_aggregates(new, 'city_id', 'purchase_amount')
tmp_df_4 = successive_aggregates(new, 'merchant_category_id', 'purchase_amount')
tmp_df_5 = successive_aggregates(new, 'merchant_id', 'purchase_amount')
tmp_df_6 = successive_aggregates(new, 'subsector_id', 'purchase_amount')
tmp_df_7 = successive_aggregates(new, 'category_2', 'purchase_amount')
tmp_df_8 = successive_aggregates(new, 'category_3', 'purchase_amount')

new_feats = new_feats.merge(tmp_df_1, on=['card_id'], how='left')
new_feats = new_feats.merge(tmp_df_2, on=['card_id'], how='left')
new_feats = new_feats.merge(tmp_df_3, on=['card_id'], how='left')
new_feats = new_feats.merge(tmp_df_4, on=['card_id'], how='left')
new_feats = new_feats.merge(tmp_df_5, on=['card_id'], how='left')
new_feats = new_feats.merge(tmp_df_6, on=['card_id'], how='left')
new_feats = new_feats.merge(tmp_df_7, on=['card_id'], how='left')
new_feats = new_feats.merge(tmp_df_8, on=['card_id'], how='left')

In [ ]:
for c in new_feats.columns[new_feats.isnull().sum() != 0].values:
    new_feats[c].fillna(new_feats[c].median(), inplace=True)

In [ ]:
new.head()

In [ ]:
new_feats.head()

### **3.2.3. time**

In [ ]:
# Features of day
new['year'] = new['purchase_date'].dt.year
new['month'] = new['purchase_date'].dt.month
new['woy'] = new['purchase_date'].dt.weekofyear
new['doy'] = new['purchase_date'].dt.dayofyear
new['wday'] = new['purchase_date'].dt.dayofweek
new['day'] = new['purchase_date'].dt.day
new['hour'] = new['purchase_date'].dt.hour
for m in ['nunique', 'mean', 'min', 'max']:
    for c in ['year', 'month', 'woy', 'doy', 'wday', 'day', 'hour']:
        new_feats['new_transac_{}_{}'.format(c, m)] = new.groupby(['card_id'])[c].agg([m]).values

# Special days in a year
new['ChristmasDay_2017'] = (pd.to_datetime('2017-12-25') - new['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0).values
new['FathersDay_2017'] = (pd.to_datetime('2017-08-13') - new['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0).values
new['ChildrenDay_2017'] = (pd.to_datetime('2017-10-12') - new['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0).values
new['BlackFriday_2017'] = (pd.to_datetime('2017-11-24') - new['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0).values
new['ValentineDay_2017'] = (pd.to_datetime('2017-06-12') - new['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0).values
new['MothersDay_2018'] = (pd.to_datetime('2018-05-13') - new['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0).values
for c in ['ChristmasDay_2017', 'FathersDay_2017', 'ChildrenDay_2017', 'BlackFriday_2017', 'ValentineDay_2017', 'MothersDay_2018']:
    new_feats['new_transac_{}_mean'.format(c)] = new.groupby(['card_id'])[c].mean().values

# Weekend
new['is_weekend'] = (new['purchase_date'].dt.weekday >= 5).astype(int)
new_feats['new_transac_purchase_weekend_count'] = new.groupby(['card_id'])['is_weekend'].sum().values
new_feats['new_transac_purchase_weekend_mean'] = new.groupby(['card_id'])['is_weekend'].mean().values

# Days difference between 2 consecutive purchase_date
new = new.sort_values('purchase_date')

new['prev_1_purchase_date'] = new.groupby(['card_id'])['purchase_date'].shift(1)
new['purchase_date_diff_days'] = (new['purchase_date'] - new['prev_1_purchase_date']).dt.days.values
new['purchase_date_diff_seconds'] = (new['purchase_date'] - new['prev_1_purchase_date']).dt.seconds.values
new['purchase_date_diff_hours'] = new['purchase_date_diff_seconds'].values // 3600
for m in ['mean', 'std', 'max', 'min']:
    new_feats['new_transac_purchase_date_diff_sec_{}'.format(m)] = new.groupby(['card_id'])['purchase_date_diff_seconds'].agg([m]).values
    new_feats['new_transac_purchase_date_diff_day_{}'.format(m)] = new.groupby(['card_id'])['purchase_date_diff_days'].agg([m]).values
    new_feats['new_transac_purchase_date_diff_hour_{}'.format(m)] = new.groupby(['card_id'])['purchase_date_diff_hours'].agg([m]).values

new['prev_2_purchase_date'] = new.groupby(['card_id'])['purchase_date'].shift(2)
new['purchase_date_diff_2_days'] = (new['purchase_date'] - new['prev_2_purchase_date']).dt.days.values
new['purchase_date_diff_2_seconds'] = (new['purchase_date'] - new['prev_2_purchase_date']).dt.seconds.values
new['purchase_date_diff_2_hours'] = new['purchase_date_diff_2_seconds'].values // 3600
for m in ['mean', 'std', 'max', 'min']:
    new_feats['new_transac_purchase_date_diff_2_sec_{}'.format(m)] = new.groupby(['card_id'])['purchase_date_diff_2_seconds'].agg([m]).values
    new_feats['new_transac_purchase_date_diff_2_day_{}'.format(m)] = new.groupby(['card_id'])['purchase_date_diff_2_days'].agg([m]).values
    new_feats['new_transac_purchase_date_diff_2_hour_{}'.format(m)] = new.groupby(['card_id'])['purchase_date_diff_2_hours'].agg([m]).values

# Days difference between the first and last purchase_date
new_feats['new_purchase_date_last'] = new.groupby(['card_id'])['purchase_date'].max().values
new_feats['new_purchase_date_first'] = new.groupby(['card_id'])['purchase_date'].min().values
new_feats['new_purchase_date_diff_day'] = (pd.to_datetime(new_feats['new_purchase_date_last']) - pd.to_datetime(new_feats['new_purchase_date_first'])).dt.days.values
new_feats['new_purchase_count_ratio'] = new_feats['new_transac_count'].values / (1. + new_feats['new_purchase_date_diff_day'].values)

# Months difference
new['month_diff'] = (pd.to_datetime('2018-12-31') - pd.to_datetime(new['purchase_date'])).dt.days // 30
new['month_diff'] += new['month_lag']
new_feats['new_month_diff_mean'] = new.groupby(['card_id'])['month_diff'].mean().values

new['duration'] = new['purchase_amount'].values * new['month_diff'].values
new['amount_month_ratio'] = new['purchase_amount'].values / (1. + new['month_diff'].values)
for m in ['mean', 'std', 'min', 'max', 'skew']:
    new_feats['new_transac_duration_{}'.format(m)] = new.groupby(['card_id'])['duration'].agg([m]).values
    new_feats['new_transac_amount_month_ratio_{}'.format(m)] = new.groupby(['card_id'])['amount_month_ratio'].agg([m]).values

# Month_lag
for m in ['mean', 'std', 'max']:
    new_feats['new_transac_monthlag_{}'.format(m)] = new.groupby(['card_id'])['month_lag'].agg([m]).values

for c in ['month_lag=1', 'month_lag=2']:
    new_feats['new_transac_{}_count'.format(c)] = new.groupby(['card_id'])[c].sum().values
    new_feats['new_transac_{}_mean'.format(c)] = new.groupby(['card_id'])[c].mean().values
new_feats['new_transac_month_lag=1_2_ratio'] = new_feats['new_transac_month_lag=1_count'].values / (1. + new_feats['new_transac_month_lag=2_count'].values)

In [ ]:
for c in new_feats.columns[new_feats.isnull().sum() != 0].values:
    new_feats[c].fillna(new_feats[c].median(), inplace=True)

In [ ]:
new.head()

In [ ]:
new_feats.head()

### **3.2.4. merchant**

In [ ]:
# Revisited transactions

# Create new_merchant table: count transactions
new_merchant = new.groupby(['merchant_id']).size().reset_index()
new_merchant.columns = ['merchant_id', 'merchant_customer_count']

# Create new_merchant_card table: count transactions for each customer
new_merchant_card = new.groupby(['merchant_id', 'card_id']).size().reset_index()
new_merchant_card.columns = ['merchant_id', 'card_id', 'customer_visit_count']

# Choose intimate customer
new_merchant_card = new_merchant_card.loc[new_merchant_card['customer_visit_count'] > 1]

# Create new_merchant_repurchase_binary table
new_merchant_repurchase_binary = new_merchant_card.groupby(['merchant_id']).size().reset_index()
new_merchant_repurchase_binary.columns = ['merchant_id', 'revisited_customers']
new_merchant_repurchase_binary['revisited_customers'].fillna(0.0, inplace=True)

# Create new_merchant_repurchase_exact table
new_merchant_repurchase_exact = new_merchant_card.groupby(['merchant_id'])['customer_visit_count'].sum().reset_index()
new_merchant_repurchase_exact.columns = ['merchant_id', 'revisited_count']
new_merchant_repurchase_exact['revisited_count'].fillna(0.0, inplace=True)

# Processing
new_merchant = new_merchant.merge(new_merchant_repurchase_binary, on=['merchant_id'], how='left')
new_merchant = new_merchant.merge(new_merchant_repurchase_exact, on=['merchant_id'], how='left')
new_merchant['repurchase_customer_ratio'] = new_merchant['revisited_customers'].values / new_merchant['merchant_customer_count'].values
new_merchant['repurchase_ratio'] = new_merchant['revisited_count'].values / new_merchant['merchant_customer_count'].values

new = new.merge(new_merchant, on=['merchant_id'], how='left')
for m in ['mean', 'std', 'max', 'min']:
    new_feats['merchant_repurchase_customer_ratio_{}'.format(m)] = new.groupby(['card_id'])['repurchase_customer_ratio'].agg([m]).values
    new_feats['merchant_repurchase_ratio_{}'.format(m)] = new.groupby(['card_id'])['repurchase_ratio'].agg([m]).values

In [ ]:
for c in new_feats.columns[new_feats.isnull().sum() != 0].values:
    new_feats[c].fillna(new_feats[c].median(), inplace=True)

In [ ]:
new.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,reference_month,month_lag=1,month_lag=2,category_2=1,category_2=2,category_2=3,category_2=4,category_2=5,category_3=0,category_3=1,category_3=2,category_3=3,new_transac_count,transac_seq_num,transac_seq_num_desc,transac_decay,transac_amount_decay,transac_month_decay,transac_amount_month_decay,year,month,woy,doy,wday,day,hour,ChristmasDay_2017,FathersDay_2017,ChildrenDay_2017,BlackFriday_2017,ValentineDay_2017,MothersDay_2018,is_weekend,prev_1_purchase_date,purchase_date_diff_days,purchase_date_diff_seconds,purchase_date_diff_hours,prev_2_purchase_date,purchase_date_diff_2_days,purchase_date_diff_2_seconds,purchase_date_diff_2_hours,month_diff,duration,amount_month_ratio,merchant_customer_count,revisited_customers,revisited_count,repurchase_customer_ratio,repurchase_ratio
0,0,C_ID_7c628841cb,69,0,1.0,1,80,M_ID_c03b62d83d,1,46.36,2017-03-01 03:24:51,1.0,9,37,2017-02,1,0,1,0,0,0,0,0,1,0,0,50,1,48,0.000022,0.001034,2.2,101.992,2017,3,9,60,2,1,3,0,0,0,0,0,0,0,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,23,1066.28,1.931667,2291,NaN,NaN,NaN,NaN
1,0,C_ID_25d399500c,19,0,1.0,1,307,M_ID_2445d76702,1,174.06,2017-03-01 11:01:06,1.0,9,19,2017-02,1,0,1,0,0,0,0,0,1,0,0,21,1,19,0.014412,2.508469,2.2,382.932,2017,3,9,60,2,1,11,0,0,0,0,0,0,0,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,23,4003.38,7.252500,88,NaN,NaN,NaN,NaN
2,0,C_ID_e3542c52f1,199,0,0.0,0,307,M_ID_b16ae63c45,1,49.93,2017-03-01 11:27:39,4.0,14,19,2017-02,1,0,0,0,0,1,0,1,0,0,0,9,1,7,0.209715,10.471080,2.2,109.846,2017,3,9,60,2,1,11,0,0,0,0,0,0,0,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,23,1148.39,2.080417,4,NaN,NaN,NaN,NaN
3,0,C_ID_fabd47ca44,69,0,0.0,0,80,M_ID_b35d0757d1,1,5.42,2017-03-01 11:55:11,1.0,9,37,2017-02,1,0,1,0,0,0,0,1,0,0,0,9,1,7,0.209715,1.136656,2.2,11.924,2017,3,9,60,2,1,11,0,0,0,0,0,0,0,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,23,124.66,0.225833,13,NaN,NaN,NaN,NaN
4,0,C_ID_6f9a771d17,96,0,1.0,1,178,M_ID_b6b9b8ed67,1,549.99,2017-03-01 12:37:26,2.0,24,29,2017-02,1,0,0,1,0,0,0,0,1,0,0,31,1,29,0.001547,0.851068,2.2,1209.978,2017,3,9,60,2,1,12,0,0,0,0,0,0,0,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,23,12649.77,22.916250,23,NaN,NaN,NaN,NaN


In [ ]:
new_feats.head()

,card_id,new_transac_count,new_transac_city_nunique,new_transac_state_nunique,new_transac_merchant_category_nunique,new_transac_subsector_nunique,new_transac_merchant_nunique,new_transac_category_1_1_count,new_transac_category_1_0_count,new_transac_category_1_1_mean,new_transac_category_1_1_std,new_transac_category_2=1_count,new_transac_category_2=1_mean,new_transac_category_2=2_count,new_transac_category_2=2_mean,new_transac_category_2=3_count,new_transac_category_2=3_mean,new_transac_category_2=4_count,new_transac_category_2=4_mean,new_transac_category_2=5_count,new_transac_category_2=5_mean,new_transac_category_3=0_count,new_transac_category_3=0_mean,new_transac_category_3=1_count,new_transac_category_3=1_mean,new_transac_category_3=2_count,new_transac_category_3=2_mean,new_transac_category_3=3_count,new_transac_category_3=3_mean,new_transac_installments_mean,new_transac_installments_sum,new_transac_installments_max,new_transac_installments_min,new_transac_installments_std,new_transac_installments_skew,new_transac_monthlag_count_std,new_transac_monthlag_count_max,new_transac_merchant_category_id_entropy,new_transac_subsector_id_entropy,new_transac_merchant_id_entropy,...,new_transac_purchase_date_diff_2_sec_std,new_transac_purchase_date_diff_2_day_std,new_transac_purchase_date_diff_2_hour_std,new_transac_purchase_date_diff_2_sec_max,new_transac_purchase_date_diff_2_day_max,new_transac_purchase_date_diff_2_hour_max,new_transac_purchase_date_diff_2_sec_min,new_transac_purchase_date_diff_2_day_min,new_transac_purchase_date_diff_2_hour_min,new_purchase_date_last,new_purchase_date_first,new_purchase_date_diff_day,new_purchase_count_ratio,new_month_diff_mean,new_transac_duration_mean,new_transac_amount_month_ratio_mean,new_transac_duration_std,new_transac_amount_month_ratio_std,new_transac_duration_min,new_transac_amount_month_ratio_min,new_transac_duration_max,new_transac_amount_month_ratio_max,new_transac_duration_skew,new_transac_amount_month_ratio_skew,new_transac_monthlag_mean,new_transac_monthlag_std,new_transac_monthlag_max,new_transac_month_lag=1_count,new_transac_month_lag=1_mean,new_transac_month_lag=2_count,new_transac_month_lag=2_mean,new_transac_month_lag=1_2_ratio,merchant_repurchase_customer_ratio_mean,merchant_repurchase_ratio_mean,merchant_repurchase_customer_ratio_std,merchant_repurchase_ratio_std,merchant_repurchase_customer_ratio_max,merchant_repurchase_ratio_max,merchant_repurchase_customer_ratio_min,merchant_repurchase_ratio_min
0,C_ID_00007093c1,2,2,2,2,2,2,0,2,0.000000,0.000000,1,0.500000,0,0.000000,1,0.50000,0,0.0,0,0.000000,0,0.000000,2,1.000000,0,0.000000,0,0,1.000000,2.0,1.0,1.0,0.000000,0.000000,1.414214,2.0,0.693147,0.693147,0.693147,...,29008.808104,7.209906,8.048957,78791.0,23.0,21.0,6891.0,2.0,1.0,2018-04-09 16:23:59,2018-04-03 11:13:35,6,0.285714,10.500000,574.665000,4.808144,35.970522,0.915435,549.23,4.160833,600.10,5.455455,1.448074,1.448109,2.000000,0.000000,2,0,0.000000,2,1.000000,0.000000,0.044339,0.088719,0.019155,0.038281,0.044339,0.088719,0.044339,0.088719
1,C_ID_0001238066,26,8,4,15,9,25,2,24,0.076923,0.271746,23,0.884615,0,0.000000,0,0.00000,0,0.0,3,0.115385,1,0.038462,21,0.807692,4,0.153846,0,0,1.653846,43.0,10.0,0.0,2.038476,3.365920,5.656854,17.0,2.414284,1.892616,3.204778,...,28076.531437,3.865258,7.815699,82676.0,13.0,22.0,2279.0,0.0,0.0,2018-04-30 19:57:30,2018-03-01 16:48:27,60,0.426230,10.153846,1180.090385,10.553191,1153.175488,10.503931,41.20,0.374545,4449.50,40.450000,1.545831,1.581470,1.346154,0.485165,2,17,0.653846,9,0.346154,1.700000,0.044339,0.088719,0.000000,0.000000,0.044339,0.088719,0.044339,0.088719
2,C_ID_0001506ef0,2,1,1,2,2,2,0,2,0.000000,0.000000,0,0.000000,0,0.000000,2,1.00000,0,0.0,0,0.000000,2,1.000000,0,0.000000,0,0.000000,0,0,0.000000,0.0,0.0,0.0,0.000000,0.000000,1.414214,2.0,0.693147,0.693147,0.693147,...,29008.808104,7.209906,8.048957,78791.0,23.0,21.0,6891.0,2.0,1.0,2018-03-22 09:14:30,2018-03-16 22:21:58,5,0.333333,10.000000,154.900000,1.408182,78.064589

# **4. Save data**

In [ ]:
new_feats_dir = '/content/drive/MyDrive/Colab Notebooks/viettel/data/new_transac.csv' # change here

In [ ]:
new_feats.to_csv(, index=False)